# The First Half of Each Student:
# Using a Markov Analysis Written in Python to Randomly Generate New Business School Course Descriptions 

By Kenneth Burchfiel

Code, Comments, and Output Released under the MIT License 

The idea for using a Markov analysis to create random text is from Think Python (2nd Edition) by Allen B. Downey. However, I did not consult Downey's solution while writing my code. Think Python can be accessed at https://greenteapress.com/wp/think-python-2e/

The course descriptions that serve as an input to the program come from the Columbia Business School website and are not my property; I'm including them here on a fair use basis.

## Introduction
Think Python, a programming textbook written by Allen B. Downey, introduced me to the concept of a Markov analysis. Markov analyses allow you to generate random, yet somewhat coherent text through an analysis of a pre-existing block of text. 

This program demonstrates how to program a Markov analysis in Python. I apply this code by using it to 'create' new courses at Columbia Business School (CBS), where I'm currently a student. The descriptions for these courses are a mashup of various CBS classes.

First, I'll import a few modules:

In [1]:
import time
start_time = time.time()

import numpy as np

Next, I'll create a function for converting a text file into a list of individual words. This will make it easier to perform a Markov analysis.

In [2]:
def text_file_to_word_list(file_name, header_length=0,footer_length=0,make_lowercase = True, keep_letters_only = True): # Turns a text file into a list with all the words in that file. header_length and footer_length represent the number of lines from the beginning and end of the text file, respectively, to exclude from the word list. For example, if you'd like to exclude the table of contents and foreword from the Markov analysis of a book, and the text of the book itself starts on line 1005, you can set header_length to 1004 to skip the table of contents and foreword. (This assumes that the first line in the text file is line 1, not line 0. Because the first line in line_list will be line 0, the line in line_list that corresponds to line 1005 in the text file will be line 1004, not 1005.)
    word_list = []
    with open(file_name) as fin: # For an explanation of why 'with open' is used, see https://docs.python.org/3/tutorial/inputoutput.html
        line_list = fin.readlines() # See https://docs.python.org/3/tutorial/inputoutput.html
    for i in range (header_length, len(line_list)-footer_length): # Uses the header_length and footer_length variables to narrow the range of line_list that will be examined. If header_length and footer_length are both 0, all of line_list will be accessed.
        # if i == header_length: # These 3 lines are useful for ensuring that the for loop starts at the right position within the text file
        #     print("i is:",line_list[i])
        #     print("i-1 is:",line_list[i-1])
        line = line_list[i]
        line_words = line.split() # Turns each line of the document into a list of strings                
        for word in line_words:
            if make_lowercase == True:
                word = word.lower()
            if keep_letters_only == True: 
                # The following lines filter out non-letter characters from each word. See https://docs.python.org/3/library/stdtypes.html
                new_string = ''
                for character in word:
                    if character.isalpha() == True:
                        new_string+= character
                if len(new_string) > 0: # Some strings (i.e. ones that were originally numbers) will have a length of 0, so they shouldn't be appended to the list. 
                    word_list.append(new_string)    
            else:
                word_list.append(word)   
    return word_list

Next, I will create two different word lists. word_list_raw contains all words in their original format, whereas word_list converts all words to lowercase and removes non-alphabetic characters. The print statements show how many words are in each list. (word_list_raw is a bit larger because it includes words that don't contain any letters; these words aren't present in word_list.)

In [3]:
word_list_raw = text_file_to_word_list('cbs_course_descriptions_ansi.txt',5,0,make_lowercase=False,keep_letters_only=False)
word_list = text_file_to_word_list('cbs_course_descriptions_ansi.txt',5,0,make_lowercase=True,keep_letters_only=True)
print(len(word_list))
print(len(word_list_raw))

15605
15653


These word lists can then be saved to files using the following code. However, because the lists produced by this program aren't terribly large, I decided not to save them to their own list.

In [4]:
# with open('word_list.txt', 'w') as f:
#     for word in word_list:
#         f.write(word) # From error message: "write() argument must be str, not list"
#         f.write(' ')


Next, I'll define the function that will perform the Markov analyses. This function creates a list of dictionaries; each dictionary stores (1) a prefix and (2) a list of all suffixes that follow the prefix. The suffix list can contain duplicates, which will help in the course of random text generation (since more frequent suffixes will be more likely to be called).

The prefix is a set of words of a given length (prefix_length); each suffix is a single word that follows that prefix.

In [5]:
def markov_analysis(word_list,prefix_length):
    prefix_suffix_dict = {}
    for i in range(len(word_list)-(prefix_length)): # The range of this for loop is truncated by (prefix_length-1) in order to prevent the for j in range . . . loop and the suffix loop from searching for words outside of the word list.
        prefix = []
        for j in range(prefix_length): # This loop will create prefixes of length prefix_length.
            prefix.append(word_list[i+j]) # Adds a new word to the prefix
        prefix_tuple = tuple(prefix) # Lists can't be keys of dictionaries, hence the need to convert them to another type (such as a tuple).
        if prefix_tuple not in prefix_suffix_dict:
            prefix_suffix_dict[prefix_tuple] = [] # This empty list will store all words that follow the prefix in the text block.
        suffix = word_list[i+prefix_length] # The suffix is simply the word that comes after the words in the prefix. 
        prefix_suffix_dict[prefix_tuple].append(suffix) # Adds the suffix to the list for which the prefix serves as the key
        # if len(prefix_suffix_dict) % 1000 == 0:
        #      print("Dictionary length is now:",len(prefix_suffix_dict)) 
    return(prefix_suffix_dict)

The following block uses the markov_analysis function to create four different dictionaries. The dictionaries with l1 at the end use single-word prefixes, and those with l2 use double-word prefixes. Meanwhile, the two dictionaries with 'raw' in their name use word_list_raw; the other two use word_list.

The print statements show how many keys are present in each dictionary. The output shows that there are many more unique 2-word prefixes than 1-word prefixes, and that the raw dictionaries contain more unique words. (This makes sense, as 'Students.' and 'students' will show up as two unique prefixes within word_list_raw, whereas in word_list, both words will be converted to the single prefix 'student.'

In [6]:
cbs_prefix_suffix_dict_l1 = markov_analysis(word_list,1)
cbs_prefix_suffix_dict_l2 = markov_analysis(word_list,2)
cbs_prefix_suffix_dict_raw_l1 = markov_analysis(word_list_raw,1)
cbs_prefix_suffix_dict_raw_l2 = markov_analysis(word_list_raw,2)


print(len(cbs_prefix_suffix_dict_l1))
print(len(cbs_prefix_suffix_dict_l2))
print(len(cbs_prefix_suffix_dict_raw_l1))
print(len(cbs_prefix_suffix_dict_raw_l2))

2825
10678
3910
11369


The keys in this dictionary are the unique one-word and two-word combinations found in the text. For instance, the list of keys in cbs_prefix_suffix_dict_raw_l1 starts with the following:

In [7]:
list(cbs_prefix_suffix_dict_raw_l1.keys())[0:5]

[('This',), ('course',), ('emphasizes',), ('the',), ('role',)]

Meanwhile, the list of keys in cbs_prefix_suffix_dict_raw_l2 starts with the following:

In [8]:
list(cbs_prefix_suffix_dict_raw_l2.keys())[0:5]

[('This', 'course'),
 ('course', 'emphasizes'),
 ('emphasizes', 'the'),
 ('the', 'role'),
 ('role', 'of')]

These keys each contain two words, whereas the keys in cbs_prefix_suffix_dict_l1 contain only one word. In both cases, however, the keys are in tuple format.

Each key has a list as its value. This list contains all the words that were found to immediately proceed that key in the text. For instance, here is the value corresponding to the ('role', 'of') key in cbs_prefix_suffix_dict_l2:

In [9]:
cbs_prefix_suffix_dict_raw_l2[('role', 'of')]

['marketing', 'the', 'the', 'technology', 'the']

In other words, 'marketing', 'the', 'the', 'technology', and 'the' are the five words that proceed 'role of' whenever it appeared in the text source. Because 'the' accounts for 3 out of 5 appearances, it has a 60% chance of being selected to follow 'role of' when random text is being produced through the random_text_using_dict function below; meanwhile, 'marketing' and 'technology' each have a 20% chance of being selected.

It's now time to use these dictionaries to generate random text. First, I'll create a random number generator using 1337 as a seed.

In [10]:
rng = np.random.default_rng(1337)

Next, I'll create a function that creates random text from a given dictionary. 

In [11]:
def random_text_using_dict(markov_dict,length): # The 'length' variable represents how many words the output should equal.
    random_text = '' # This string will store the output of the function.
    for i in range(length):
        if i == 0:
            random_key = rng.choice(list(markov_dict.keys())) # The first key (i.e. the first prefix) in the dictionary is chosen at random. It will be a list since markov_dict.keys() is converted to a list before rng.choice() is applied. New values of random_key that appear later will also be in list format.
            random_text += ' '.join(random_key) # The prefix is then added to random_text.
            prefix_length = len(random_key) # The length of the first key is stored for use in a for loop later in this function. Since the first prefix will be the same length as all other prefixes, the prefix length only needs to be checked once.
            # print("prefix_length is:",prefix_length)
        # print("random_key is:",random_key)
        random_value = rng.choice(markov_dict[tuple(random_key)]) # Next, a random value (i.e. a random suffix, or word following the prefix) is selected among all the suffixes for that value of random_key. For instance, if the random_key was "Students will" and the values in the dictionary for that prefix were "learn", "discuss", and "experience", random_value could be any one of those 3 entries.
        random_text += ' ' + random_value # Note that, after the first prefix is added to random_text, all subsequent text additions will be suffixes (or 'random_value' values). Adding the prefixes (random_key values) would result in redundant text.
        new_random_key = [] # The next key 
        if prefix_length == 1:
            new_random_key.append(random_value) # If the prefixes in this dictionary are only 1 word long, random_value will become the new key. Appending random_value to new_random_key ensures that the result remains a list, which is important when checking whether this key is present in markov_dict.keys() later on.
        else: # Otherwise, new_random_key will be created by combining all words in random_key following the first word with random_value. For example, if the current prefix is "Students will discuss", and the current value is "the", new_random_key will become "will discuss the".
            for i in range (1, prefix_length): # If the length of the prefix is instead longer than 1, this loop will append all words in the prefix other than the first to new_random_key. Afterwards, random_value will be appended, generating a new prefix.
                new_random_key.append(random_key[i])
            new_random_key.append(random_value)
        random_key = new_random_key 
        # print("random_key before check:",random_key)
        if tuple(random_key) not in markov_dict.keys(): # If random_key was based on the end of the text, that key may not be present in the dictionary, which would lead to a KeyError being raised. In this case, a new random key is chosen.
            # print("Key not found")
            random_key = rng.choice(list(markov_dict.keys()))

    return random_text


Now it's time to see what these 'new' business school courses look like! The following code blocks apply random_text_using_dict() to the four different dictionaries created earlier. 

You'll notice a couple patterns in the output. First, the dictionaries ending in l1 (indicating that the length of their prefix is only 1) produce less coherent (but perhaps more entertaining) results. That's because having only one word as the prefix produces more suffix options, so it's easier for the text to jump from one part of the text block to another. The two-prefix dictionaries (those ending in l2) are a bit more coherent, since it's harder for them to jump to other subjects.

Second, the output from the 'raw' dictionaries (which include spacing and punctuation) is a bit more readable because the commas and periods offer pauses and breaks in the text.

In [12]:
random_course_summary_l1 = random_text_using_dict(cbs_prefix_suffix_dict_l1,100)
random_course_summary_l1

'desired ends achieved the analytics and the course will give you this will endure even as one makes in groups analyze a sport in many wish to rank players in sales marketing has market and management class discussion on the information content covered in the th century signal sea changes in hockey league baseball football league baseball etc class we explore both from direct advice experience in the retail such novel technologies students with an overview of this course is composed of government and apply these concepts to manage group project students who are internal values clarified how to identify attractive'

In [13]:
random_course_summary_l2 = random_text_using_dict(cbs_prefix_suffix_dict_l2,100)
random_course_summary_l2

'relevant theories and subject them to gain advantage in the face of disruption by analyzing the regulatory process navigating the halls of power and the role of the course will address all topics in the world of political economy laid by the rapid changes in american political economy during the course is based will be offered starting in the innovation salon students will conduct in groups company analyses for digital marketing has become an essential skill for those interested in a project class participation and a onepage investment circular for introductions launch your startup focuses on design and product management in some'

In [14]:
random_course_summary_raw_l1 = random_text_using_dict(cbs_prefix_suffix_dict_raw_l1,100)
random_course_summary_raw_l1

'conclude with digital businesses. In this class to senior-level consultant, knowing how to the asset management. This seminar seeks to better understanding to address a working through partner and political economy (and not in the acute current players in other institutions and growth. What makes a regulatory environments affect international sports businesses. Students will be used to the semester course extends and focus on human-machine teams, the requisite analytical techniques that is Mandatory. The purpose of story structure and trends. In this time and Presidential candidates debate how these assets of the implications of commercial real estate debt sources to come'

In [15]:
random_course_summary_raw_l2 = random_text_using_dict(cbs_prefix_suffix_dict_raw_l2,100)
random_course_summary_raw_l2

'Given this trend, it is important to how to use retail data and trend spotting to predict future performance of players or teams? How to predict future sales and trends. In this course, students will conduct an in-depth look at how specific elements of the course will be exposed to the impact on future success, both personally and professionally. While the course examines the decision one makes in security analysis and operational decision making, with emphasis on how organizations change, and how the top right side of a problem as too much. Conflict is a course designed to be made. The primary'

You may be interested in what the output from a dictionary using 3-word prefixes looks like. I didn't find these outputs to be as interesting; my hypothesis is that the reduced number of alternative suffixes for 3-word prefixes made it harder for the text to jump from one course to another.

In [16]:
cbs_prefix_suffix_dict_raw_l3 = markov_analysis(word_list_raw,3)
random_course_summary_raw_l3 = random_text_using_dict(cbs_prefix_suffix_dict_raw_l3,100)
random_course_summary_raw_l3


'world. The course is focused on teaching students the foundational basics of markets systems, mechanics of traditional sales and trading, and the concept of market liquidity. Students will then progress to learning about the role of the merchant, the basics of programming variables, strings, lists, functions, and all that stuff but we\'ll be doing it with a focus on business use cases. You\'ll learn how to "rip apart" a company and draw thoughtful conclusions about whether it might make for a compelling short "pitch." Additionally, students will learn about a wide range of operations topics, including process flow analysis, supply chain management, capacity'

Finally, I'll create two text files that each contain 10 randomly generated course descriptions. The first file, ten_new_courses_l1.txt, will use cbs_prefix_suffix_dict_raw_l1 (the one-word-prefix raw dictionary) as its source, whereas the second file, ten_new_courses_l2.txt, will use cbs_prefix_suffix_dict_raw_l2 (the two-word-prefix raw dictionary) as its source.

In [22]:
with open('ten_new_courses_l1.txt', 'w') as f:
    for i in range(10):
        course_string = ("Course "+str(i+1)+":\n"+random_text_using_dict(cbs_prefix_suffix_dict_raw_l1,100)+'\n\n')
        f.write(course_string) # From error message: "write() argument must be str, not list"

In [20]:
with open('ten_new_courses_l2.txt', 'w') as f:
    for i in range(10):
        course_string = ("Course "+str(i+1)+":\n"+random_text_using_dict(cbs_prefix_suffix_dict_raw_l2,100)+'\n\n')
        f.write(course_string) # From error message: "write() argument must be str, not list"



I ran these loops a number of times and saved some of my favorite examples in 'notable_descriptions.txt'. I hope you enjoy reading them! For instance, the title at the top of this notebook derives from one of the examples. It reads, in full:


"Python is used for pricing and risk inherent in many business problems. As a manager seeking to maximize the synergy between theory and applications, corporate control and recapitalizations. The classes are structured to maximize firm value, understanding your market(s) is crucial to achieving your goals. The first half of each student will select one of the current marketplace, and the work load may be structured as for-profit, a hybrid for-profit/nonprofit enterprise, or be a demanding class meant for the evaluation methods are meant to simulate as close as possible to the latest academic research on fundamental analysis. The course has a direct"

What does the second half of each student do, then? We may never know.


Finally, I'll calculate how long it took the program to run.

In [19]:
end_time = time.time()

run_time = end_time - start_time
run_minutes = int(run_time // 60)
run_seconds = run_time % 60
print("Total run time:",'{:.2f}'.format(run_time),"second(s) ("+str(run_minutes),"minute(s) and",'{:.2f}'.format(run_seconds),"second(s))") # Only produces an accurate result when the program is run nonstop from start to finish


Total run time: 0.96 second(s) (0 minute(s) and 0.96 second(s))


That's it! As you can see, a Markov analysis is not too difficult to implement, and it's fun to experiment with different prefix lengths. I hope you enjoy creating your own random text using other code blocks.